# Fine-tuning a model with the Trainer API Chinese

安裝套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece] --quiet
!pip install accelerate -U --quiet
!pip install wandb --quiet

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: josh-hu. Use `wandb login --relogin` to force relogin


True

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("sepidmnorozy/Chinese_sentiment")
checkpoint = "google-bert/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/2591 [00:00<?, ? examples/s]

In [5]:
tokenizer(tokenized_datasets['train']['text'][0])

{'input_ids': [101, 6983, 2421, 2791, 7279, 5050, 679, 7097, 8024, 1920, 2414, 2523, 5653, 6900, 511, 3241, 677, 4717, 4638, 2523, 7676, 8024, 6983, 2421, 4472, 1862, 679, 1168, 1724, 3215, 3560, 3976, 8024, 852, 1348, 3683, 676, 3215, 7770, 511, 1765, 7953, 2523, 1962, 8024, 1019, 7092, 6900, 704, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenized_datasets['train']['input_ids'][0]

[101,
 6983,
 2421,
 2791,
 7279,
 5050,
 679,
 7097,
 8024,
 1920,
 2414,
 2523,
 5653,
 6900,
 511,
 3241,
 677,
 4717,
 4638,
 2523,
 7676,
 8024,
 6983,
 2421,
 4472,
 1862,
 679,
 1168,
 1724,
 3215,
 3560,
 3976,
 8024,
 852,
 1348,
 3683,
 676,
 3215,
 7770,
 511,
 1765,
 7953,
 2523,
 1962,
 8024,
 1019,
 7092,
 6900,
 704,
 511,
 102]

### 定義`Trainer`的參數
我們定義我們之前的第一步`Trainer`是定義一個類別，其中包含將用於訓練和評估的`TrainingArguments`所有超參數。為求簡單，這邊`Trainer`提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", num_train_epochs=10, report_to="wandb")

使用`AutoModelForSequenceClassification`有兩個標籤的類別

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

### 定義`Trainer`
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、`training_args`、訓練和驗證資料集、`data_collator`和分詞器

In [9]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### 說明
這樣傳遞分詞器時，訓練器使用的預設`data_collator`將是之前定義的`DataCollatorWithPadding`，因此可以跳過`data_collator=data_collator`這一行

### 開始訓練
呼叫`Trainer`中的`train()`方法

In [ ]:
trainer.train()

Step,Training Loss
500,0.471400
1000,0.452200
1500,0.433800
2000,0.394400
2500,0.396700
3000,0.383400
3500,0.408900
4000,0.533400
4500,0.438700
5000,0.403300


### 改進的地方
開始微調之後並每500步報告一次訓練損失。但不會知道模型的表現如何，因為有幾個原因
* 沒有設定`evaluation_strategy`為`steps`（每`eval_steps`評估一次）或`epoch`（在每個`epoch`結束時評估）來告訴訓練器在訓練期間進行評估
* 沒有為訓練器提供`compute_metrics()`函數，在上述評估期間計算指標

### 對模型這行評估
使用validation資料集，讓資料集通過訓練好的模型。由於validation資料中有408筆資料，每筆資料經過模型之後，會有兩個logits，所以是408x2個。第一個是label為0的浮點數，第二個是label為1的浮點數，較大的值對應到模型預測出來可能性較高的結果。

另外`labels.ids`就是ground true真值，不是0就是1，所以也有408x1個

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
print(predictions.predictions[0])
print(predictions.label_ids[0])

也可以用softmax表示

In [ ]:
import numpy as np

# Defining the softmax function
def softmax(x):
    e_x = np.exp(x - np.max(x))  # Subtracting the max for numerical stability
    return e_x / e_x.sum()

# Input array
x = np.array(predictions.predictions[0])

# Applying the softmax function
softmax_values = softmax(x)
softmax_values

In [ ]:
import torch
import torch.nn.functional as F

x_tensor = torch.tensor(predictions.predictions[0])
softmax_values_torch = F.softmax(x_tensor, dim=0)
softmax_values_torch

predict() 方法的輸出是另一個命名組合，包含三個欄位：predictions、label_ids 和 metrics。metrics 欄位將只包含傳遞的資料集上的損失，以及一些時間度量（預測所需的總時間和平均時間）。一旦完成我們的 compute_metrics() 函式並將其傳遞給 Trainer，該欄位也將包含 compute_metrics() 返回的度量。

predictions 是一個形狀為 408 x 2 的二維陣列（408 是我們使用的資料集中的元素數量）。這些是我們傳遞給 predict() 的資料集中每個元素的 logits, 所有 Transformer 模型都返回 logits）。為了將它們轉換成我們可以與我們的標籤比較的預測，我們需要取第二軸上最大值的索引

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
print("預測：", preds[:20])
print("真值：", predictions.label_ids[:20])

### 計算模型表現的度量方法
有很多種度量模型表現的指標，如408個預測中，有幾個是和直值一模一樣的，然後算出不一樣的比例，這種稱為accuracy，也有其它的度量方法，如f1 score等等。針對不同資料集，會有不同的度量方法，我們只要載入那個資料集對對的度量方法即可。

使用`Evaluate` 函式庫的度量。我們可以像載入資料集一樣輕鬆地載入與 MRPC 資料集相關的度量，這次使用 `evaluate.load()` 函式。返回的物件有一個 `compute()` 方法，使用它來進行度量計算：

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

定義一個`compute_metrics()`方法作為參數傳入`Trainer`的物件，就可以在訓練時列出每一個階段的accuracy和f1 score了。

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer",
                  evaluation_strategy="epoch",
                  num_train_epochs=10,
                  report_to="wandb")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()